In [1]:
from datetime import datetime, date, timedelta
import pandas as pd
import numpy as np
import glob
import os
from tqdm import tqdm
import re

In [2]:
df_contests = pd.read_csv("contest_data-2022-02-01-to-2022-10-03.csv")

In [3]:
df_contests['start_date'] = pd.to_datetime(df_contests['start_time']).dt.date.apply(str)

In [4]:
df_users = pd.read_csv('user_level_change.csv')
df_out = pd.DataFrame()
time_interval = 28

for idx, row in df_users.iterrows():
    user_id = row['user_id']
    date_of_change = row['date_of_change']

    date_of_change = datetime.strptime(date_of_change, '%Y-%m-%d')
    start_date = str((date_of_change - timedelta(days=time_interval)).date())
    end_date = str((date_of_change + timedelta(days=time_interval)).date())
    start_date, end_date

    df_contests_select = df_contests[(df_contests['start_date'] >= start_date) &  (df_contests['start_date'] <= end_date) ]
    df_contests_select.shape


    ## find all contests the solver joined 
    pattern =  "'participant_id': '" + str(user_id) + "'"
    df_contests_select = df_contests_select[(df_contests_select['entries'].str.contains(pattern)) | (df_contests_select['deleted_entries'].str.contains(pattern))]
    df_contests_select['after_change'] = df_contests_select['start_date'].apply(lambda x: 1 if str(x) >= str(date_of_change) else 0)
    df_contests_select.shape

    ## get entry id
    entries = []
    pattern = "'entry-\d{1,3}', 'participant_id': '" + str(user_id) + "'"
    for i, r in df_contests_select.iterrows():
        e = []
        e += re.findall(pattern, r['entries'])
        e += re.findall(pattern, r['deleted_entries'])
        e = list(map(lambda x: x.split(',')[0], e))
        entries.append(e)

    df_contests_select['submissions']= entries
    df_contests_select['id'] = user_id

    df_out = pd.concat([df_out, df_contests_select])


In [5]:
df_out.shape

(2474, 36)

In [6]:
df_out

,id,title,url,seeker,start_time,name_to_incorporate,slogan_to_incorporate,description,industry,colors_to_explore,...,top_level,avg_contest_won,avg_contest_won_top_5,avg_contest_won_top_10,avg_runner_up,avg_runner_up_top_5,avg_runner_up_top_10,start_date,after_change,submissions
3361,1555767,Jasper & Partners logo,https://99designs.hk/logo-design/contests/jasp...,sbinkevicM,"Wed, 16 Mar 2022 17:31:15 +0000",Jasper & Partners,Helping you actualize your potential,"This company does resume, essay and grant writ...",Business & Consulting,"['Reds', 'Oranges', 'Designers choose']",...,0.0,114.500000,131.8,114.5,275.500000,303.8,275.5,2022-03-16,0,['entry-27']
3769,1555767,Design a logo for an online fitness company wh...,https://99designs.hk/logo-design/contests/desi...,AWALSH5,"Wed, 16 Mar 2022 20:52:17 +0000",Andrew Walsh Fitness,The way you do one thing is the way you do eve...,Andrew Walsh Fitness is a completely online ba...,Physical Fitness,"['Yellows', 'Dark neutrals', 'Greens']",...,2.0,75.380952,216.2,144.0,190.809524,543.2,363.0,2022-03-16,0,"['entry-32', 'entry-31', 'entry-35', 'entry-34..."
7572,3531389,Blue Frog Logo,https://99designs.hk/logo-design/contests/blue...,weiler_to2,"Fri, 03 Jun 2022 05:14:30 +0000",Blue Frog,Unspecified,Blue Frog is a Holding organisation that doesn...,Accounting & Financial,['Designers choose'],...,4.0,56.242424,152.4,120.7,147.696970,478.8,334.6,2022-06-03,0,"['entry-12', 'entry-11']"
7777,3531389,Happy Man Trucking Logo,https://99designs.hk/logo-design/contests/happ...,beekeeper3,"Fri, 03 Jun 2022 03:51:10 +0000",Happy Man Trucking,Unspecified,"We transport gravel, rock, sand, compost, blac...",Landscaping,"['Yellows', 'Greens', 'Reds']",...,1.0,52.428571,107.4,69.4,118.714286,268.6,160.8,2022-06-03,0,"['entry-34', 'entry-33', 'entry-9', 'entry-8',..."
8442,3531389,A podcast for a life coach,https://99designs.hk/logo-design/contests/podc...,moislam83,"Mon, 13 Jun 2022 19:03:08 +0000",Thrive with Noha,Unspecified,A life coaching podcast to unleash the power w...,Entertainment & The Arts,['Designers choose'],...,2.0,51.315789,142.2,84.7,150.157895,422.6,255.8,2022-06-13,0,"['entry-12', 'entry-2', 'entry-1']"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11627,2737898,"Logo update- modern, designer vibe!",https://99designs.hk/logo-design/contests/logo...,mgnF,"Mon, 22 Aug 2022 15:45:14 +0000",Myranda Ayla Designs,see below .. I need help on a tag.,please look up my website..\nhttp://www.myrand...,Architectural,"['Greens', 'Dark neutrals', 'Light neutrals']",...,1.0,43.761905,113.4,80.2,128.809524,332.0,234.8,2022-08-22,1,"['entry-8', 'entry-7', 'entry-6']"
11907,2737898,Oort Space industries,https://99designs.hk/logo-design/contests/oort...,morarú™,"Thu, 25 Aug 2022 03:22:09 +0000",Oort,space industries,Space exploration technologies & Off world exp...,Technology,"['Dark neutrals', 'Designers choose', 'Light n...",...,7.0,32.835165,163.8,125.3,86.857143,380.2,322.0,2022-08-25,1,"['entry-249', 'entry-79']"
11717,4805920,Eden (Paradies),https://99designs.hk/logo-design/contests/eden...,chritho7,"Fri, 19 Aug 2022 10:59:01 +0000",Eden,Unspecified,wir bauen Luxusvillen die an einem Golfressort...,Architectural,"['Greens', 'Aquas', 'Blues']",...,3.0,57.589744,264.2,172.4,197.692308,975.4,619.0,2022-08-19,0,['entry-40']
11866,4805920,Design a Welcoming Logo for our Hawaiian Gift ...,https://99designs.hk/logo-design/contests/desi...,laurenceysm,"Sun, 21 Aug 2022 09:09:13 +0000",Happy Place,Jewelry & Gifts,Happy Place is a family-owned jewelry & gift s...,Retail,"['Blues', 'Yellows', 'Designers choose']",...,21.0,56.764151,282.4,230.0,175.566038,905.0,730.6,2022-08-21,0,"['entry-362', 'entry-359']"
